In [8]:
!pip install html2text

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=af187fe8ca5aa9aba20968445ca5ffb234f15e7db400f944bfeb37a7fb5cbbaf
  Stored in directory: /Users/rj/Library/Caches/pip/wheels/2b/01/23/578505d65e2a97d78bf1fe3fc8256ecf37572dc1df598b0eaf
Successfully built html2text

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import requests
from bs4 import BeautifulSoup
import html2text
import mygene
import json
import pickle
mg = mygene.MyGeneInfo()

In [ ]:
parts_to_remove = [
    "##  Summary\n",
    "NEW",
    'Try the newGene table',
    'Try the newTranscript table',
    '**',
    "\nGo to the top of the page Help\n"
]

def rough_text_from_gene_name(gene_number):
    
    # get url
    url = f"https://www.ncbi.nlm.nih.gov/gene/{gene_number}"
    # Send a GET request to the URL
    summary_text = ''
    soup = None
    try:
        response = requests.get(url, timeout=30)
    except requests.exceptions.Timeout:
        print('time out')
        return((summary_text,soup))
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the "summary" tab content by inspecting the page's structure
        summary_tab = soup.find('div', {'class': 'rprt-section gene-summary'})

        # Check if the "summary" tab content is found
        if summary_tab:
            # Convert the HTML to plain text using html2text
            html_to_text = html2text.HTML2Text()
            html_to_text.ignore_links = True  # Ignore hyperlinks

            # Extract the plain text from the "summary" tab
            summary_text = html_to_text.handle(str(summary_tab))
            # Remove the specified parts from the original text
            for part in parts_to_remove:
                summary_text = summary_text.replace(part, ' ')
                # Replace '\n' with a space
            summary_text = summary_text.replace('\n', ' ')

            # Reduce multiple spaces into one space
            summary_text = ' '.join(summary_text.split())
            # Print or save the extracted text
        else:
            print("Summary tab not found on the page.")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    return((summary_text,soup))

In [ ]:
# Using Gene CD24 as an example
cd_24_name = mg.querymany('CD24', scopes='symbol', species='human')

In [ ]:
gene_name_to_tax_id = {}
for result in cd_24_name:
    if "_id" in result and "query" in result:
        gene_name_to_tax_id[result['symbol']] = result['_id']

In [ ]:
gene_name_to_tax_id

In [ ]:
gene_name_to_summary_page = {}

In [ ]:
for gene_name, page_id in sorted(gene_name_to_tax_id.items()):
    if gene_name not in gene_name_to_summary_page:
        print('gene_name',gene_name)
        parsed_text, unparsed_html = rough_text_from_gene_name(page_id)
        gene_name_to_summary_page[gene_name] = parsed_text

In [ ]:
gene_name_to_summary_page

### Experiment with Varied Gene Sets Tailored to Your Needs
#### For any specific gene name, mygene can be utilized to translate it into page IDs. We've illustrated an example using gene vocabularies in scGPT and Geneformer. Download links for these files are available in the repository's README.

In [ ]:

from src.utils import download_gdrive_file

scgpt_vocab_url = "https://drive.google.com/file/d/1H3E_MJ-Dl36AQV6jLbna2EdvgPaqvqcC/view?usp=drive_link"
scgpt_vocab_path = "data/vocab.json"
download_gdrive_file(scgpt_vocab_url, scgpt_vocab_path)
vocab_gene = json.load(open(scgpt_vocab_path, 'rb'))
vocab_gene_list = list(vocab_gene.keys())

In [4]:
from src.utils import download_file
import pickle

geneformer_token_dict_url = "https://huggingface.co/ctheodoris/Geneformer/resolve/main/geneformer/token_dictionary_gc95M.pkl"
geneformer_token_dict_path = "data/token_dictionary.pkl"
download_file(geneformer_token_dict_url, geneformer_token_dict_path)
token_dictionary = pickle.load(open(geneformer_token_dict_path, 'rb'))

File already exists at data/token_dictionary.pkl


In [10]:

# example query to convert gene IDs into page ids for NCBI 
vocab_gene_list_results = mg.querymany(sorted(vocab_gene_list), scopes='symbol', species='human')
token_dictionary_results = mg.querymany(sorted(token_dictionary.keys()), fields="symbol")

6975 input query terms found dup hits:	[('A2ML1-AS1', 2), ('A2ML1-AS2', 2), ('A2MP1', 2), ('AACSP1', 2), ('AADACL2-AS1', 3), ('AADACP1', 2)
21975 input query terms found no hit:	['5S_rRNA_ENSG00000276861', '5S_rRNA_ENSG00000277411', '5S_rRNA_ENSG00000277488', '5S_rRNA_ENSG00000
7 input query terms found dup hits:	[('ENSG00000267635', 2), ('ENSG00000268674', 3), ('ENSG00000275921', 2), ('ENSG00000278277', 3), ('E
11 input query terms found no hit:	['<cls>', '<eos>', '<mask>', '<pad>', 'ENSG00000139656', 'ENSG00000168078', 'ENSG00000189144', 'ENSG
